In [1]:
import sys
import random

DPLL and WalkSAT algo on the CNF from TP1

    c simple_v3_c2.cnf
    c
    p cnf 3 2
    1 -3 0
    2 3 -1 0

In [51]:
# DPLL

clauses, nvars, lit_clause = parse('in.cnf')
solution = backtracking(clauses, [])
  
if solution:
    solution += [x for x in range(1, nvars + 1) if x not in solution and -x not in solution]
    solution.sort(key=lambda x: abs(x))
    print ('SATISFIABLE:')
    print (' '.join([str(x) for x in solution]))
else:
    print ('UNSATISFIABLE')

SATISFIABLE:
1 2 3


In [38]:
# WalkSAT

clauses, nvars, lit_clause = parse('in.cnf')
solution = run_sat(clauses, nvars, lit_clause)

if solution:
    print ('SATISFIABLE')
    print (' '.join(map(str, solution[1:])))
else:
    print ('UNSATISFIABLE')

SATISFIABLE
1 2 3


## DPLL

Require: CNF formula F, empty model M

Ensure: UNSAT or SAT with model M

    procedure DPLL(F, M)
        (F,M) ← UnitPropagate(F,M)
        if All clauses are true in M then
            return SAT
        end if
        if One clause is wrong in M then
            return UNSAT
        end if
        
        
        l ← choose a literal not assigned in M
        if DPLL(F,M ∪ {l}) = SAT then
            return SAT
        end if
         return DPLL(F¬l,M ∪ {¬l})
    end procedure
    
    procedure UNITPROGAGATE(F, M)
        while F contains no empty clause but has a unit clause K with literal l do
            F ← Fl
            M ← M ∪ {l}
        end while
         return (F,M)
    end procedure

L'algorithme prend en entrée une formule de la logique propositionnelle en forme normale conjonctive. L'algorithme est basé sur une méthode de backtracking. Il procède en choisissant un littéral, lui affecte une valeur de vérité, simplifie la formule en conséquence, puis vérifie récursivement si la formule simplifiée est satisfaisable. Si c'est le cas, la formule originale l'est aussi, dans le cas contraire, la même vérification est faite en affectant la valeur de vérité contraire au littéral. Dans la terminologie de la littérature DPLL, c'est la conséquence d'une règle dite splitting rule (règle de séparation), et sépare le problème en deux sous problèmes.

L'étape de simplification consiste essentiellement en la suppression de toutes les clauses rendues vraies par l'affectation courante, et tous les littéraux déduits faux à partir de l'ensemble des clauses restantes.

DPLL étend l'algorithme de backtracking par l'utilisation des deux règles suivantes :

La propagation unitaire
Si une clause est unitaire, c'est-à-dire qu'elle contient un et un seul littéral, elle ne peut être satisfaite qu'en affectant l'unique valeur qui la rend vraie à son littéral. Il n'y a par conséquent plus à choisir. En pratique, son application entraîne une cascade d'autres clauses unitaires de manière déterministe, et évite donc d'explorer une grande partie de l'espace de recherche. Elle peut être vue comme une forme de propagation de contraintes.
L'élimination des littéraux dits purs
Si une variable propositionnelle apparaît seulement sous forme positive ou seulement sous forme négative alors ses littéraux sont dits purs. Les littéraux purs peuvent être affectés d'une manière qui rend toutes les clauses qui les contiennent vrais. Par conséquent ces clauses ne contraignent plus l'espace de recherche et peuvent être éliminées.
L'incohérence d'une affectation partielle des variables est détectée quand une clause devient vide, c'est-à-dire quand les littéraux de la clause sont tous faux. La satisfiabilité d'une formule est déterminée quand toutes les variables sont affectées sans qu'une clause ne devienne vide, ou bien, dans les implémentations modernes de l'algorithme, quand toutes les clauses sont satisfaites. L'incohérence de la formule complète ne peut être déterminée qu'après une recherche exhaustive.

In [2]:
# Read DIMACS files

def parse(filename):
    clauses = []
    count = 0
    for line in open(filename):

        if line[0] == 'c':
            continue
        if line[0] == 'p':
            n_vars = int(line.split()[2])
            lit_clause = [[] for _ in range(n_vars * 2 + 1)]
            continue

        clause = []
        for literal in line[:-2].split():
            literal = int(literal)
            clause.append(literal)
            lit_clause[literal].append(count)
        clauses.append(clause)
        count += 1
    return clauses, int(n_vars), lit_clause

In [3]:
# Propagation

def bcp(cnf, unit):
    modified = []
    for clause in cnf:
        if unit in clause: continue
        if -unit in clause:
            c = [x for x in clause if x != -unit]
            if len(c) == 0: return -1
            modified.append(c)
        else:
            modified.append(clause)
    return modified


In [4]:
# Count literals

def get_counter(formula):
    counter = {}
    for clause in formula:
        for literal in clause:
            if literal in counter:
                counter[literal] += 1
            else:
                counter[literal] = 1
    return counter


In [5]:
# Affect pure literal

def pure_literal(formula):
    counter = get_counter(formula)
    assignment = []
    pures = []
    for literal, times in counter.items():
        if -literal not in counter: pures.append(literal)
    for pure in pures:
        formula = bcp(formula, pure)
    assignment += pures
    return formula, assignment

In [6]:
# Propagate unit

def unit_propagation(formula):
    assignment = []
    unit_clauses = [c for c in formula if len(c) == 1]
    while len(unit_clauses) > 0:
        unit = unit_clauses[0]
        formula = bcp(formula, unit[0])
        assignment += [unit[0]]
        if formula == -1:
            return -1, []
        if not formula:
            return formula, assignment
        unit_clauses = [c for c in formula if len(c) == 1]
    return formula, assignment

In [7]:
# Select literal

def variable_selection(formula):
    counter = get_counter(formula)
    return random.choice(list(counter.keys()))

In [8]:
def backtracking(formula, assignment):
    formula, pure_assignment = pure_literal(formula)
    formula, unit_assignment = unit_propagation(formula)
    assignment = assignment + pure_assignment + unit_assignment
    if formula == - 1:
        return []
    if not formula:
        return assignment

    variable = variable_selection(formula)
    solution = backtracking(bcp(formula, variable), assignment + [variable])
    if not solution:
        solution = backtracking(bcp(formula, -variable), assignment + [-variable])
    return solution

## WalkSAT

 function WalkSAT(clauses, p, max-flips) returns a satisfying model or failure

    inputs: clauses, a set of clauses in propositional logic
            p, the probability of choosing to do a “random walk” move, typically around 0.5
            max-flips, number of flips allowed before giving up
    model ←a random assignment of true/false to the symbols in clauses
    for i = 1 to max-flips do
        if model satisfies clauses then return model
        clause ←a randomly selected clause from clauses that is false in model
        with probability p flip the value in model of a randomly selected symbol
        
 from clause

        else flip whichever symbol in clause maximizes the number of satisfied clauses
    return failure

In [10]:
def get_random_interpretation(n_vars):
    return [i if random.random() < 0.5 else -i for i in range(n_vars + 1)]

In [11]:
def get_true_sat_lit(clauses, interpretation):
    true_sat_lit = [0 for _ in clauses]
    for index, clause in enumerate(clauses):
        for lit in clause:
            if interpretation[abs(lit)] == lit:
                true_sat_lit[index] += 1
    return true_sat_lit

In [12]:
def update_tsl(literal_to_flip, true_sat_lit, lit_clause):
    for clause_index in lit_clause[literal_to_flip]:
        true_sat_lit[clause_index] += 1
    for clause_index in lit_clause[-literal_to_flip]:
        true_sat_lit[clause_index] -= 1

In [16]:
def compute_broken(clause, true_sat_lit, lit_clause, omega=0.4):
    break_min = sys.maxsize
    best_literals = []
    for literal in clause:

        break_score = 0

        for clause_index in lit_clause[-literal]:
            if true_sat_lit[clause_index] == 1:
                break_score += 1

        if break_score < break_min:
            break_min = break_score
            best_literals = [literal]
        elif break_score == break_min:
            best_literals.append(literal)

    if break_min != 0 and random.random() < omega:
        best_literals = clause

    return random.choice(best_literals)

In [17]:
def run_sat(clauses, n_vars, lit_clause, max_flips_proportion=4):
    max_flips = n_vars * max_flips_proportion
    while 1:
        interpretation = get_random_interpretation(n_vars)
        true_sat_lit = get_true_sat_lit(clauses, interpretation)
        for _ in range(max_flips):

            unsatisfied_clauses_index = [index for index, true_lit in enumerate(true_sat_lit) if
                                         not true_lit]

            if not unsatisfied_clauses_index:
                return interpretation

            clause_index = random.choice(unsatisfied_clauses_index)
            unsatisfied_clause = clauses[clause_index]

            lit_to_flip = compute_broken(unsatisfied_clause, true_sat_lit, lit_clause)

            update_tsl(lit_to_flip, true_sat_lit, lit_clause)

            interpretation[abs(lit_to_flip)] *= -1
